In [42]:
import os
import openai
import json

from dotenv import load_dotenv
load_dotenv() 

openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": "Hi my name is Earl, how are you?"}
    ],
    functions=[

        {
            "name": "translateToSpanish",
            "description": "Translate to spanish",
            "parameters": {
                'title': 'StepByStepAIResponse',
                'type': 'object',
                'properties': {'english': {'title': 'english', 'type': 'string'},
                               'spanish': {'title': 'spanish', 'type': 'string'}},
                'required': ['english', 'spanish']
            }
        }
    ],
    function_call={"name": "translateToSpanish"}
)

In [ ]:
print(res.choices[0]["message"]["function_call"]["arguments"])

first step is to get intent or http verb
extract the target
query the database for target
extract params
build the api call
confirmwith the user
call the api
display api call status

Here I can extract the HTTP VERB from the context, might need to adjust to work well with UKG's APIs

In [37]:
res = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "system",
                "content": "Only respond with the following http methods get, post, put or delete based on the user response"},
            {"role": "user", "content": "terminate john dowe"}
        ],
        functions=[

            {
                "name": "extractHttpMethod",
                "description": "only return get, post, put or delete",
                "parameters": {
                    'title': 'HTTP Method Response',
                    'type': 'object',
                    'properties': {'method': {'title': 'method', 'type': 'string'}},
                    'required': ['method']
                }
            }
        ],
        function_call={"name": "extractHttpMethod"}
    )
print(res.choices[0]["message"]["function_call"]["arguments"])


{
  "method": "delete"
}


In [24]:
def promptToHTTPVerb(prompt):
    return eval(openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "assistant",
                "content": "Only respond in HTTP Verbs such as GET, POST, PUT and DELETE"},
            {"role": "user", "content": prompt}
        ],
        functions=[

            {
                "name": "promptToHTTPVerb",
                "description": "Extract the http verb from the string and return only a single http verb",
                "parameters": {
                    'title': 'Single AI Response',
                    'type': 'object',
                    'properties': {'verb': {'title': 'verb', 'type': 'string'}},
                    'required': ['verb']
                }
            }
        ],
        function_call={"name": "promptToHTTPVerb"}
    ).choices[0]["message"]["function_call"]["arguments"])["verb"]


promptToHTTPVerb("test")

'test'

Here I can extract the subject of the users reply

In [9]:
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role":"assistant","content":"Only respond with the name of the subject of the message."},
        {"role": "user", "content": "My name is Kunta Kinte, Who is John Snow?"}
    ],
    functions=[
        {
            "name": "extractSubject",
            "description": "Extract the subject of a users reply",
            "parameters": {
                'title': 'Single AI Response',
                'type': 'object',
                'properties': {'subject': {'title': 'subject', 'type': 'string'}},
                'required': ['subject']
            }
        }
    ],
    function_call={"name": "extractSubject"}
)
print(res.choices[0]["message"]["function_call"]["arguments"])

{
  "subject": "John Snow"
}


In [10]:
def extractSubject(prompt):
    return openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "assistant",
                "content": "Only respond with the name of the subject of the message."},
            {"role": "user", "content": "My name is Kunta Kinte, Who is John Snow?"}
        ],
        functions=[
            {
                "name": "extractSubject",
                "description": "Extract the subject of a users reply",
                "parameters": {
                    'title': 'Single AI Response',
                    'type': 'object',
                    'properties': {'subject': {'title': 'subject', 'type': 'string'}},
                    'required': ['subject']
                }
            }
        ],
        function_call={"name": "extractSubject"}
    ).choices[0]["message"]["function_call"]["arguments"]


Here I try to generate the params fo send a get request to the UKG API

In [ ]:
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "system", "content": "Only respond with the name of the subject of the message."},
        {"role": "user", "content": "My name is Kunta Kinte, Who is John Snow?"}
    ],
    functions=[
        {
            "name": "generateParamsObject",
            "description": "Generate an object that stores the firstName and LastName",
            "parameters": {
                'title': 'Single AI Response',
                'type': 'object',
                'properties': {
                    'firstName': {'title': 'firstName', 'type': 'string'}, 
                    'lastName': {'title': 'lastName', 'type': 'string'}
                    },
                'required': ['firstName', 'lastName']
            }
        }
    ],
    function_call={"name": "generateParamsObject"}
)
print(res.choices[0]["message"]["function_call"]["arguments"])


In [45]:
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role":"system","content":'''Only respond with 'true' or 'false' given a user message.
         if user response is nope, then respond false,if user response is no, then respond false,'''},
        {"role": "user", "content": "heck no"}
    ],
    functions=[
        {
            "name": "trueOrFalse",
            "description": "return 'true' if response is positive or affirmative, otherwise return 'false' if response is not positive or not affirmative",
            "parameters": {
                'title': 'true/false response',
                'type': 'object',
                'properties': {'boolean': {'title': 'boolean', 'type': 'boolean'}},
                'required': ['boolean']
            }
        }
    ],
    function_call={"name": "trueOrFalse"}
)
print(type(json.loads(res.choices[0]["message"]["function_call"]["arguments"])['boolean']))

<class 'bool'>


In [11]:
def postRoute(chats):
    subject =  extractSubject
    print('this worked')

In [13]:
chats = [
    {"role": "system", "content": "you are a helpful assistant"},
    {"role": "assistant", "content": "How may I help you"},
]

prompt = input("What do you want")
chats.append({"role": "user", "content": prompt})

verb = promptToHTTPVerb(prompt)
print(verb)

if verb == 'GET':
    pass
elif verb == 'POST':
    postRoute()
elif verb == 'PUT':
    pass
elif verb == 'DELETE':
    pass
else:
    pass



{
  "verb": "PUT"
}


In [46]:
res = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role":"assistant","content":"rephrase the user response and ask them if they would like to proceed"},
        {"role": "user", "content": "update john snow to jon dowe"}
    ],
    functions=[
        {
            "name": "summarize",
            "description": "summarize the user response and ask if they would like to proceed",
            "parameters": {
                'title': 'AI response',
                'type': 'object',
                'properties': {'summary': {'title': 'summary', 'type': 'string'}},
                'required': ['summary']
            }
        }
    ],
    function_call={"name": "summarize"}
)
print(res.choices[0]["message"]["function_call"]["arguments"])

{
  "summary": "You would like to update John Snow to Jon Dowe. Would you like to proceed?"
}


In [48]:
def ask_to_proceed(prompt):
    return json.loads(openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "assistant",
                "content": "rephrase the user response and ask them if they would like to proceed"},
            {"role": "user", "content": prompt}
        ],
        functions=[
            {
                "name": "summarize",
                "description": "summarize the user response and ask if they would like to proceed",
                "parameters": {
                    'title': 'AI response',
                    'type': 'object',
                    'properties': {'summary': {'title': 'summary', 'type': 'string'}},
                    'required': ['summary']
                }
            }
        ],
        function_call={"name": "summarize"}
    ).choices[0]["message"]["function_call"]["arguments"])["summary"]


ask_to_proceed("update john snow to jon dowe")

'You want to update John Snow to Jon Dowe. Would you like to proceed?'

In [51]:
def generate_variations(prompt):
    return json.loads(openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "assistant",
                "content": "generate a variation of the users response by rephrasing the response"},
            {"role": "user", "content": prompt}
        ],
        functions=[
            {
                "name": "variations",
                "description": "generate a variation of the users response",
                "parameters": {
                    'title': 'variation of response',
                    'type': 'object',
                    'properties': {'variation': {'title': 'variation', 'type': 'string'}},
                    'required': ['variation']
                }
            }
        ],
        function_call={"name": "variations"}
    ).choices[0]["message"]["function_call"]["arguments"])["variation"]


generate_variations("I am the best in the world")

'I am the greatest in the world'

In [57]:
def field_to_update(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "assistant",
                "content": "User the users response and extract only one pythonic field name that the user would like to change "},
            {"role": "user", "content": prompt}
        ],
        functions=[
            {
                "name": "field_extractor",
                "description": "extract the most lifely pythonic field name that the user would like to modify",
                "parameters": {
                    'title': 'user field snake case',
                    'type': 'object',
                    'properties': {'field': {'title': 'field', 'type': 'string'}},
                    'required': ['field']
                }
            }
        ],
        function_call={"name": "field_extractor"}
    ).choices[0]["message"]["function_call"]["arguments"]
    return json.loads(response)["field"]


field_to_update("change john dowe's last name to john snow")

'last_name'